In [ ]:
# Install and load environment variables from .env file
%pip install python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded successfully")

## setup

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m pip install --upgrade pip

In [4]:
%pip install google-genai

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pymupdf
%pip install markdownify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pathlib
import textwrap
import google.generativeai as genai
#from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import pandas as pd

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Load API key from environment variable
api_key = os.getenv("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_GENAI_API_KEY not found in environment variables. Please check your .env file.")

#genai.configure(api_key=api_key)

In [8]:
import IPython.display
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [ ]:
# Use the environment variable for Project ID
import os
import vertexai

PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
if not PROJECT_ID:
    raise ValueError("GOOGLE_CLOUD_PROJECT not found in environment variables. Please check your .env file.")

LOCATION = os.getenv("GOOGLE_CLOUD_REGION", "asia-northeast1")

vertexai.init(project=PROJECT_ID, location=LOCATION)


In [10]:
MODEL_ID = 'gemini-2.5-flash'

In [11]:
def get_prompt():
    prompt = """
    ## 依頼

    あなたはODAレポートのテキストマイニングに精通したAIアシスタントです。
    テキストから、次の情報を抽出し、Pythonで扱いやすいようにJSONで返してください。
    
    1. title: プロジェクト名
    2. conclusion: 結論・もしくは総合評価欄に記載されている文章
    3. validity: 妥当性・整合性欄に記載されている文章
    4. inpact: 有効性・インパクトに記載されている文章
    5. effectiveness: 効率性に記載されている文章  
    6. sustainability: 持続性に記載されている文章 
    7. background: 事業の背景に記載されている文章
    8. description: 事業の目的と実施内容に記載されている文章
    9. description_detail: 事業の背景、事業の目的、実施内容に記載されている文章を要約(500字以内)

    ## 注意事項
    • 各フィールドは500文字を上限に抽出すること。  
    • なるべく原文のまま抽出すること。
    • JSON のキー以外の余計な文字は出力しないでください。
    • 評価書に書かれていることのみ書いてください。記載がない場合は "-"を入力してください。
    • 各フィールドについて、評価そのものがわかる文章は書かないでください。例えば、「よって、総合評価は高い」といった文章は抽出しないでください。
    
    出力例：
    ```json
    {
      "title": "...",
      "conclusion": "... ～ ...",
      "validity": "... ～ ...",
      "inpact": "... ～ ...",
      "effectiveness": "... ～ ...",
      "sustainability": "... ～ ...",
      "background": "... ～ ...",
      "description": "... ～ ...",
    }
    """
    return prompt

In [13]:
# お試し実行
from google import genai
import httpx
from google.genai import types

client = genai.Client(api_key=api_key)
pdf_file_uri = "https://www2.jica.go.jp/ja/evaluation/pdf/2020_0900816_4_f.pdf"

doc_data = httpx.get(pdf_file_uri).content

prompt = get_prompt()
pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents=[
  types.Part.from_bytes(
    data=doc_data,
    mime_type='application/pdf',
  ), prompt
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
)
print(response.text)

```json
{
  "title": "案件別事後評価(内部評価)評価結果票:技術協力プロジェクト",
  "conclusion": "本事業はフェーズ1とフェーズ2から構成されており、RHD(現 RMNCAH 局)及びPRHOのRH行政運営能力改善というプロジェクト目標を一部達成した。事業効果は事業完了後も一部継続している。特に RMNCAH 局のプログラム管理能力や研修能力の向上という効果が継続し、母子保健サービスの質の向上という上位目標は達成された。持続性については、実施機関の制度・体制面、技術面、財務面において、人員不足や予算不足などの問題がみられた。効率性については、事業費が計画を若干上回った。",
  "validity": "フェーズ1の事前評価時、本事業は公衆衛生省が2002年に発表した基本方針「基本的な保健サービスを広く平等に提供及び六つの優先分野(1)5歳未満児死亡率の削減、2)妊産婦死亡率の削減、3)栄養不良の抑制、4)感染症の抑制、5) 保健医療サービス、6)効果的かつ効率的な保健医療サービスを提供するために必要な人材の能力向上)に合致していた。フェーズ2の事前評価時は、本事業は「アフガニスタン国家開発戦略」(2008年~2013年)がリプロダクティブヘルス・子どもの健康プログラムに高い優先度を置いていることと合致していた。上記「事業の背景」に記したとおり、本事業はフェーズ1とフェーズ2それぞれの事前評価時、中央及び地方の RHO の能力向上という開発ニーズと合致していた。フェーズ1の事前評価時である 2004年時点での日本の対アフガニスタン支援において、保健分野への支援は三つの重点分野の一つ「復興・復旧支援」の小分野として行われていた。フェーズ2の事前評価時には、2009年7月の経済協力政策協議に基づき、日本政府は「教育及び保健・医療等の基礎生活分野」を含む四つの重点支援分野を発表した。",
  "inpact": "プロジェクト目標は、事業完了時までに一部達成された。フェーズ1期間中には、「国家RH戦略」(2006年~2009年)を実行に移すための「国家 RH プログラム」(モニタリングやサポーティブスーパービジョンを含む)の策定やIST メカニズムの提案といったアウトプットが産出されたことから、RHDの能力が向上したといえる。もっ

In [14]:
pdf_file_uri = "https://www2.jica.go.jp/ja/evaluation/pdf/2019_0603827_4_f.pdf"

doc_data = httpx.get(pdf_file_uri).content

pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents=[
  types.Part.from_bytes(
    data=doc_data,
    mime_type='application/pdf',
  ), prompt
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
)
print(response.text)

```json
{
  "title": "技術協力プロジェクト「結核対策プロジェクト」および「結核対策プロジェクトフェーズ2」",
  "conclusion": "本事業は NTP のプログラム管理能力の強化を通じ、アフガニスタン全国で多剤耐性結核を含む結核菌検査の実施と質の高い直接監視下短期化学療法 DOTS が行われることにより、結核の有病率と死亡率の減少、つまりアフガニスタン MDGs の達成と結核による負担の軽減を目指すものであった。本事業は同国の開発政策、開発ニーズ、日本の援助政策と十分に合致しており、妥当性は高い。本事業の実施により一定の効果発現がみられ、「結核対策プロジェクト」では NTP の能力強化と全国の検査体制を整備し、DOTS を基本とする治療体制を整備した。「結核対策プロジェクトフェーズ2」では NTP のプログラム管理能力が強化され、多剤耐性結核の検査体制を整備し、地方住民・難民・受刑者へのサービスを強化した。本事業のプロジェクト目標はおおむね達成したといえる。上位目標の指標の「結核の有病率と死亡率の減少」の達成状況は数値的にはプロジェクトとの因果関係を示すことは困難だが、プロジェクト目標の達成により患者の負担が軽減されたと言え、本事業は上位目標達成へ一定程度の貢献をしたと判断される。本事業は事業費・事業期間ともに計画を上回ったため、効率性は中程度である。本事業の政策・政治的関与、制度・体制、技術、財務、いずれも問題なく、本事業によって発現した効果の持続性は高い。",
  "validity": "フェーズ1計画時、アフガニスタン公衆衛生省は結核を含めた感染症の減少を優先順位の最も高い開発課題として捉え、基本的な保健事業戦略を実施していた。また、公衆衛生省は結核などの感染症による有病率や死亡率を減少させるために3年間の中期戦略を策定していた。フェーズ1完了時とフェーズ2計画時には、アフガニスタン MDGs6のターゲット11に「2020年までに結核の有病率と死亡率を半減する」と掲げられており、結核対策は優先的な課題となっていた。また、「国家結核対策プログラムの戦略計画(2006~2010年、2009~2013年)」は、MDGs 及びストップ結核パートナーシップの目標にも合致しており、DOTS を通した質の高い結核対策サービスの提供をめざし

## 取得対象の設定

In [15]:
input_path = "../df_check_99.csv"

df_html = pd.read_csv(input_path)
df_html["URL"] = df_html["file"]
df_html

,Unnamed: 0,file,country,eval_year,evaluator_group,evaluator,project_cost_plan,project_cost_act,プロジェクト期間開始_計画時,プロジェクト期間終了_計画時,...,貿易_flg_fix,農業一般_flg_fix,通信・放送一般_flg_fix,都市計画・土地造成_flg_fix,金融_flg_fix,開発_flg_fix,cost_rate,duration_rate,total_eval,URL
0,0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ケニア,2010,OPMAC株式会社,小林信行,1.110900e+09,1.019900e+09,2002-01-01,2005-11-20,...,False,False,False,False,False,False,0.918084,0.875000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
1,1,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ベトナム,2010,グローバルリンクマネージメント株式会社,三浦順子,2.235000e+09,2.206000e+09,2002-07-01,2006-01-01,...,False,False,False,False,False,False,0.987025,1.000000,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
2,2,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,モンゴル,2010,財団法人国際開発高等教育機構,浜岡真紀,NaN,NaN,2002-06-17,2003-06-15,...,False,False,False,False,False,False,NaN,2.461538,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
3,3,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,中華人民共和国,2010,アイ・シー・ネット株式会社,岸野優子,1.785000e+09,1.718000e+09,2003-03-01,2008-01-29,...,False,False,False,False,False,False,0.962465,0.950000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
4,4,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ベトナム,2010,グローバルリンクマネージメント株式会社,三浦順子,2.277000e+09,2.271000e+09,2003-05-01,2006-03-01,...,False,False,False,False,False,False,0.997365,1.000000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2222,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,モンゴル,2024,モンゴル事務所,NaN,4.000000e+08,2.630000e+08,2016-04-01,2020-03-31,...,False,False,False,False,False,False,0.657500,1.000000,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2223,2223,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,マダガスカル,2024,マダガスカル事務所,NaN,4.000000e+08,4.090000e+08,2017-01-01,2018-12-07,...,False,False,False,True,False,False,1.022500,1.375000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2224,2224,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,ヨルダン,2024,ヨルダン事務所,NaN,3.030000e+08,2.950000e+08,2017-02-01,2020-02-01,...,False,False,False,False,False,False,0.973597,1.000000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2225,2225,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,ナイジェリア,2024,ナイジェリア事務所,NaN,2.000000e+08,2.160000e+08,2014-12-01,2016-05-31,...,False,False,False,False,False,False,1.080000,2.578947,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...


In [16]:
df_html

,Unnamed: 0,file,country,eval_year,evaluator_group,evaluator,project_cost_plan,project_cost_act,プロジェクト期間開始_計画時,プロジェクト期間終了_計画時,...,貿易_flg_fix,農業一般_flg_fix,通信・放送一般_flg_fix,都市計画・土地造成_flg_fix,金融_flg_fix,開発_flg_fix,cost_rate,duration_rate,total_eval,URL
0,0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ケニア,2010,OPMAC株式会社,小林信行,1.110900e+09,1.019900e+09,2002-01-01,2005-11-20,...,False,False,False,False,False,False,0.918084,0.875000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
1,1,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ベトナム,2010,グローバルリンクマネージメント株式会社,三浦順子,2.235000e+09,2.206000e+09,2002-07-01,2006-01-01,...,False,False,False,False,False,False,0.987025,1.000000,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
2,2,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,モンゴル,2010,財団法人国際開発高等教育機構,浜岡真紀,NaN,NaN,2002-06-17,2003-06-15,...,False,False,False,False,False,False,NaN,2.461538,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
3,3,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,中華人民共和国,2010,アイ・シー・ネット株式会社,岸野優子,1.785000e+09,1.718000e+09,2003-03-01,2008-01-29,...,False,False,False,False,False,False,0.962465,0.950000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
4,4,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ベトナム,2010,グローバルリンクマネージメント株式会社,三浦順子,2.277000e+09,2.271000e+09,2003-05-01,2006-03-01,...,False,False,False,False,False,False,0.997365,1.000000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2222,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,モンゴル,2024,モンゴル事務所,NaN,4.000000e+08,2.630000e+08,2016-04-01,2020-03-31,...,False,False,False,False,False,False,0.657500,1.000000,3.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2223,2223,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,マダガスカル,2024,マダガスカル事務所,NaN,4.000000e+08,4.090000e+08,2017-01-01,2018-12-07,...,False,False,False,True,False,False,1.022500,1.375000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2224,2224,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,ヨルダン,2024,ヨルダン事務所,NaN,3.030000e+08,2.950000e+08,2017-02-01,2020-02-01,...,False,False,False,False,False,False,0.973597,1.000000,4.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2225,2225,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,ナイジェリア,2024,ナイジェリア事務所,NaN,2.000000e+08,2.160000e+08,2014-12-01,2016-05-31,...,False,False,False,False,False,False,1.080000,2.578947,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...


In [17]:
len(df_html)

2227

## API呼び出し


In [18]:
def jsonize(jsonlikewords):
    import re
    import json

    # JSON文字列に変換
    json_text = jsonlikewords.replace("json", "")
    json_text = json_text.replace("\\n", "")
    json_text = json_text.replace("```", "")
    json_text = json_text.replace("    ", "")
    json_text = json_text.replace(",\n}", "\n}")
    

    # JSONとして読み込み
    json_data = json.loads(json_text)

    return json_data

In [19]:
import time
from google.api_core import retry
from google.api_core import exceptions as api_core_exceptions
import google.generativeai as genai
import google.genai.errors as genai_errors # ★ genai のエラーをインポート
import httpx
import logging

# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# リトライすべき google.api_core または google.genai のサーバーエラー/リソース枯渇エラーか判定する関数
def is_retryable_genai_server_error(exc):
    """google.api_core または google.genai のリトライすべき例外か判定"""
    # google.api_core の例外をチェック (500, 503, 504, 429)
    if isinstance(exc, (api_core_exceptions.InternalServerError,
                       api_core_exceptions.ServiceUnavailable,
                       api_core_exceptions.DeadlineExceeded,
                       api_core_exceptions.ResourceExhausted)):
        logging.warning(f"Detected google.api_core retryable error ({type(exc).__name__}). Retrying...")
        return True

    # ★ google.genai.errors の例外をチェック
    # ServerError (500, 503 など) をリトライ対象に含める
    if isinstance(exc, genai_errors.ServerError):
        # ServerError は 5xx 系エラーを示すため、基本的にリトライ対象とする
        logging.warning(f"Detected google.genai.errors.ServerError. Retrying...")
        return True
    # ResourceExhaustedError (429) もリトライ対象に含める
    if isinstance(exc, genai_errors.ResourceExhaustedError):
         logging.warning(f"Detected google.genai.errors.ResourceExhaustedError (Quota Exceeded?). Retrying...")
         return True
    # 他にもリトライしたい genai のエラーがあればここに追加
    # 例: if isinstance(exc, genai_errors.DeadlineExceededError): return True

    return False

# リトライすべき httpx のエラーか判定する関数
def is_retryable_http_error(exc):
    """httpx の 5xx系 または 429 HTTPエラーかどうかを判定"""
    if isinstance(exc, httpx.HTTPStatusError):
        status_code = exc.response.status_code
        is_5xx_error = status_code >= 500 and status_code < 600
        is_429_error = status_code == 429 # Too Many Requests
        if is_5xx_error or is_429_error:
             logging.warning(f"Detected HTTP Error (Status: {status_code}). Retrying...")
             return True
    return False

# 総合的なリトライ判定関数
def should_retry(exc):
    """リトライすべき例外かどうかを総合的に判断"""
    # is_retryable_genai_server_error で True ならリトライ
    if is_retryable_genai_server_error(exc):
        return True
    # is_retryable_http_error で True ならリトライ
    if is_retryable_http_error(exc):
        return True
    # google.api_core の RetryError (内部リトライからのエラー)
    if isinstance(exc, api_core_exceptions.RetryError):
         logging.warning(f"Google API Core RetryError detected. Retrying...")
         return True

    # 上記以外はリトライしない
    # Non-retryable のログレベルを WARNING に変更（エラーではないため）
    logging.warning(f"Non-retryable error occurred: {type(exc).__name__} - {exc}")
    return False


In [20]:
@retry.Retry(
    predicate=should_retry,      # リトライ条件を指定する関数
    initial=1.0,               # 最初のリトライまでの遅延時間（秒）(google.api_core の推奨は initial)
    maximum=60.0,              # 最大遅延時間（秒）(google.api_core の推奨は maximum)
    multiplier=2.0,            # 遅延時間の倍率 (google.api_core の推奨は multiplier)
    deadline=300.0,            # 全体の試行の最大時間（秒） (google.api_core の推奨は deadline)
    # on_error コールバックでリトライ時に追加処理も可能
    # on_error=lambda exc: logging.warning(f"Retrying due to {exc}")
)
def generate_content_with_retry(url: str, model_id: str):
    """
    PDFデータとプロンプトを使用してGemini APIを呼び出し、コンテンツを生成する（リトライ付き）。

    Returns:
        Gemini APIからのレスポンスオブジェクト。

    Raises:
        google.api_core.exceptions.RetryError: 指定されたリトライ回数を超えても成功しなかった場合。
        Exception: その他の予期しないエラー。
    """
    try:
        logging.info(f"Attempting to generate content with model: {model_id}")

        doc_data = httpx.get(url).content
        pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
        contents=[
          types.Part.from_bytes(
            data=doc_data,
            mime_type='application/pdf',
          ), prompt
        ]

        response = client.models.generate_content(
            model=model_id,
            contents=contents,
        )

        logging.info("Successfully generated content.")
        return response

    except (api_core_exceptions.GoogleAPIError, httpx.HTTPStatusError) as e:
        # should_retry で判定されるので、ここではログ出力のみ行い再raiseする
        # (should_retryでFalseが返れば、このraiseが最終的なエラー送出となる)
        logging.error(f"Error during API call or data processing: {e}")
        raise e
    except Exception as e:
        # 予期しないその他のエラー
        logging.exception(f"An unexpected error occurred in generate_content_with_retry: {e}")
        # 予期しないエラーはリトライさせずに上位に投げる
        raise

In [21]:
import pandas as pd
import json
import grpc
import time
from google.api_core.exceptions import InvalidArgument

prompt = get_prompt()
error=[]
path = "df_eval_sentences"
#df_out = pd.read_csv(f'output/{path}.csv')
df_out = pd.DataFrame()
j = 0

for i,row in df_html.iterrows():
    j += 1
    print(f"processing {j} out of {len(df_html)}")
    # httpの場合ダウンロードできないため置換
    f = row["URL"]
    f = f.replace("http:", "https:")

    retry_flg = len(df_out) > 0
    skip_flg = False

    if retry_flg:
      skip_flg = f in df_out["file"].values

    if skip_flg:
        print(f"already processed {f}")
    else:
        try:
          pdf_file_name = f.split('/')[-1]
          print(f"processing {f}")

          response = generate_content_with_retry(url=f,model_id=MODEL_ID)

          json_out = jsonize(response.text)
          df_data = {
              'title': [json_out['title']],
              'conclusion': [json_out['conclusion']],
              'validity': [json_out['validity']],
              'inpact': [json_out['inpact']],
              'effectiveness': [json_out['effectiveness']],
              'sustainability': [json_out['sustainability']], 
              'background': [json_out['background']], 
              'description': [json_out['description']], 
              'description_detail': [json_out['description_detail']], 
          }
          df = pd.DataFrame(df_data)
          df["file"] = f
          df_out=pd.concat([df_out, df])
          if j % 50 == 0:
              df_out.to_csv(f"../out_csv/{path}.csv")
              print(f"save df succeessed {pdf_file_name}")

        except api_core_exceptions.RetryError as e:
          logging.error(f"Failed to generate content after multiple retries: {e}")
        except Exception as e:
          logging.error(f"An error occurred during the process: {e}", exc_info=True) # トレースバックも出力

        except grpc.RpcError as e:
          error.append(f)
          print(f"failed RpcError {f}")

        except InvalidArgument:
          print(f"too many token {f}")

        except(ValueError):
          error.append(f)
          print(f"failed ValueError {f}")

2025-10-13 13:42:59,116 - INFO - Attempting to generate content with model: gemini-2.5-flash
2025-10-13 13:42:59,230 - INFO - HTTP Request: GET https://www2.jica.go.jp/ja/evaluation/pdf/2010_0200600_4_f.pdf "HTTP/1.1 200 OK"


processing 1 out of 2227
processing https://www2.jica.go.jp/ja/evaluation/pdf/2010_0200600_4_f.pdf


2025-10-13 13:42:59,375 - INFO - AFC is enabled with max remote calls: 10.


KeyboardInterrupt: 

In [74]:
df_out

,title,conclusion,validity,inpact,effectiveness,sustainability,background,description,description_detail,file
0,アティ橋・イクサ橋架け替え計画,本事業で実施したアティ橋、イクサ橋の架け替えは既存道路網の維持管理を重視するケニアの開発政策...,事前評価時における国家開発計画は、過去10年以上にわたり進行した道路インフラの劣化は不十分な...,本事業は落橋のリスクに対応し、円滑な交通を確保することが主な目的であるため、交通量と通行可能...,本事業の主なアウトプットである橋梁に関しては計画通りに建設されており、事業効果に影響を与える...,事前評価時点から事後評価時点にかけて、ケニアの道路当局は大幅に再編され、道路省が長期計画を策...,アフリカ東部に位置するケニアは1999年の段階で約15万キロの道路網が整備されており、国内物...,ケニア東部州において国道B7号線の二橋梁(アティ橋、イクサ橋)を架け替えることにより、安全で...,ケニアは1999年時点で約15万キロの道路網を持ち、国内物流の9割が道路輸送に依存。モンバサ...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
1,北部地下水開発計画,本事業は計画時、事業評価時点共に、ベトナム国の開発政策、開発ニーズとの整合性が高く、計画時の...,計画時におけるベトナム国の地方給水・衛生セクターの基本政策である「国家地方給水・衛生戦略 2...,本事業の実施は、計画時想定された、1)水因性疾病罹患率の減少、2)水汲み労働の軽減、に貢献し...,"日越双方ほぼ計画通りのアウトプットであり、予定通りの給水能力(7,490 ㎡/日)の施設が完...",ニンビン省とタンホア省ではCPC、タイグエン省ではP-CERWASSが給水施設の運営・維持管...,本事業対象地域は農村部に属し、2001年時点で公共給水施設が皆無であり、住民は浅井戸地下水や...,タイグエン省4コミューン、ニンビン省3コミューン、タンホア省5コミューン、計 12 コミュー...,本事業対象地域は、2001年時点で公共給水施設が皆無の農村部であり、住民は浅井戸や地表水を利...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
2,第二次初等教育施設整備計画,本事業は、対象10校において初等教育施設及び機材を整備することにより、教室の過密状況を緩和し...,本事業の目的は、事前評価時(2002年)、事後評価時(2011年)においてモンゴル国の国家開...,各学校への質問票回答や現地視察、受益者調査の結果から、以下のインパクトが確認された。学校が近...,日本側のアウトプットについては、軽微な設計変更はあったが、計画通りに対象10校において初等教...,本事業により整備された施設・機材の維持管理体制に問題はない。各学校に機材・設備担当者が配置さ...,モンゴルは、1990年代以降、民主化と市場経済化推進に対応できる人的資源開発を最重要課題とし...,本事業は、対象10校において初等教育施設及び機材を整備することにより、教室の過密状況を緩和し...,モンゴルでは1990年代以降の市場経済化に伴う財政悪化により教育予算が削減され、教職員・教育...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
3,第二次黄河中流域保全林造成計画,本事業は、中国政府の森林被覆率の改善を目標とした森林造成計画の中で、自然環境、立地条件ともに...,本協力対象事業は、中国政府の森林被覆率の改善を目標とした森林造成計画の中で、自然環境・立地条...,事業対象4県4地区の森林被覆率は、事業実施前の8.3%から評価時の66.4%へ大幅に向上し、...,本事業のアウトプットはほぼ計画通り実施された。植林面積は計画通りであり、ソフトコンポーネント...,実施機関である山西省林業庁は、事業完了後も組織体制に大きな変更はなく、運営維持管理体制に問題...,中国では、木材生産のための森林伐採や、食糧確保のための過剰な開墾が進み、森林は国土面積の 1...,山西省昕水河流域(吉県、大寧県、蒲県、隰県)において植林技術普及のモデルとなる森林を造成する...,本事業は、中国で進行する森林減少と、特に黄河中流域の黄土高原における深刻な土砂流出問題に対応...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
4,麻疹ワクチン製造施設建設計画,本事業の実施は、ベトナムの開発政策及び日本の援助政策と合致している。開発ニーズに関しては、計...,本事業の実施はベトナムの開発政策及び日本の援助政策と合致している。開発ニーズに関しては、計画...,計画時、本事業の実施は、1) 麻疹ワクチン2回接種対象者数、接種者数及び接種率の維持、2)乳...,本事業は事業費及び事業期間ともにほぼ計画通りであり効率性は高い。,1)過去半年の間に異常・逸脱時の対応や製造原価削減のための変更管理が適切に行われなかった事例...,ベトナム社会主義共和国(以下、ベトナム)政府は、乳児死亡率、5 歳未満児死亡率の低減及び感染...,ベトナム国ハノイ市において、ベトナム国唯一の麻疹ワクチン製造施設および品質管理施設の建設、製...,ベトナム政府は感染症対策として予防接種拡大計画を実施し、ワクチン自給を目指す中、麻疹ワクチン...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
...,...,...,...,...,...,...,...,...,...,...
2296,アンタナナリボ・トアマシナ経済都市軸(TaToM) 総合開発計画策定プロジェクト,本事業は、アンタナナリボ都市圏、トアマシナ都市圏、ならびにTaToM経済都市軸運輸・地域の開...,本事業は、事前評価時点におけるマダガスカルの開発政策および開発ニーズと整合性が高く、計画/ア...,事後評価時点において、本事業の目標はおおむね計画どおりに達成され、TaToM総合開発計画を含...,事業費はわずかに計画を上回り(計画比:102%)、事業期間は計画を上回った(計画比:138%...,【政策面】「マダガスカルのビジョン 2030年」は、グランエスト・センターを設立し産業・サー...,マダガスカルの首都アンタナナリボは、国内総生産の約30%を占める政治経済の中心地である。また...,本事業は、首都アンタナナリボ都市圏とトアマシナ都市圏のPUDi、および両都市間の運輸整備・開...,マダガスカルの政治経済の中心地であるアンタナナリボと、国内最大の商業港トアマシナは、国道2号...,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2297,若年層へのキャリアカウンセリング能力向上プロジェクト,本事業は、プロジェクト目標を、計画を超えて達成し、パイロット EOs/CGOs が提供する若...,本事業は、事前評価時点におけるヨルダンの開発政策と整合性が高い。ヨルダン政府は「国家雇用戦略...,事業完了時までに、プロジェクト目標は計画を超えて達成された。すべてのパイロットEOsが目標を...,事業費及び事業期間は計画内に収まった(計画比:それぞれ97%、100%)。アウトプットは計画...,職業技能開発委員会(TVSDC)によって新設されたキャリアカウンセリングの資格認定制度は、キ...,ヨルダン統計局によると、2016年の20歳から24歳の失業率は33.3%と非常に高く、若者が...,本事業は、パイロット EOs/CGOs において、キャリアカウンセリングの産官学連携体制の構...,ヨルダンでは20-24歳の失業率が33.3%と高く、若者の雇用問題が喫緊の課題であり、JIC...,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2298,電力マスタープラン策定プロジェクト,本事業は、送電網整備計画の礎となるマスタープランを策定した。事業完了後、電力開発のためにマス...,本事業は、「7つのアジェンダ」や「ナイジェリア・ビジョン 20:2020年」など、事前評価時...,事業完了時までに、事業目的は一部達成された。「電力マスタープラン」(2014年~2038年)...,事業費はわずかに計画を上回り(計画比:108%)、事業期間は計画を大幅に上回った(計画比:2...,FMPは国民議会でマスタープランの正式承認を進めるも、新型コロナの影響で手続きは中断。しかし...,ナイジェリアでは、発電・送電・配電の容量不足から全土で計画停電が恒常化しており、電力分野は経...,本事業は、電力需要予測、電源開発、系統整備、投資計画などを含む25年間の電力マスタープランを...,ナイジェリアでは恒常的な計画停電が発生し、電力安定供給が経済成長の阻害要因となっていた。政府...,https://www2.jica.go.jp/ja/evaluation/pdf/2024...
2299,税務局人材育成システム向上プロジェクト,本事業は、STS 職員の業務能力の向上(プロジェクト目標)をおおむね計画どおりに達成し、税務...,【事前評

In [22]:
df_out.to_csv(f"../out_csv/{path}.csv")

OSError: Cannot save file into a non-existent directory: '../out_'

In [76]:
len(df_out['file'].unique())

2301